CSE 391 Final Project - Stock Market Prediction
<br>
Team Members: Hannah Yao, Jonathan Yin, and Arjun Rao
<br>
<br>
The intention of this notebook is to simply assess various basic machine learning models' scores.
<br>
Seeing successful various models from sklearn are will help us determine which types of models are good, and what models might be worth further pursuing.

# Setup/Imports

In [208]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

import numpy as np
import pandas as pd

# Cleaning & Processing the Data

In [209]:
# ------------------ Read and Process Data ------------------ #

# I am replacing outliers instead of dropping
df = pd.read_csv("stockdata.csv", index_col='date', parse_dates=['date'])

# Replace signal outliers with last observation
sig_val = df['signal'].value_counts()
criteria = df[ (df['signal'] <= 2) | (df['signal'] >= 400) ]
df['signal'][criteria.index] = np.nan
df['signal'].fillna(method='ffill', inplace=True)

# Replace spy_close_price outliers with last observation.
# Carry last observation forward method (thing in stocks)
sig_val = df['spy_close_price'].value_counts()
criteria = df[ df['spy_close_price'] >= 500 ]
df['spy_close_price'][criteria.index] = np.nan
df['spy_close_price'].fillna(method='ffill', inplace=True)


X = df[['signal']]
Y = df['spy_close_price']
#We split using 75% of the data for training and 25% for testing
X_train, X_test , Y_train , Y_test = train_test_split(X,Y,test_size=0.25,random_state=34)
print("shapes of our train and test data:")
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

shapes of our train and test data:


((500, 1), (500,), (167, 1), (167,))

# Regressors!
<br> Note that, for each of these, we're just printing out the score. A summary of the scores is in the last cell. We are just looking to see what models are good. Our actual models for the final project are described in detail in our other ipynb files with R^2 values, etc.

In [210]:
# Logistic Regression

linreg = LinearRegression()
linreg.fit(X_train, Y_train)
Y_pred = linreg.predict(X_test)
score_lin = linreg.score(X_test, Y_test)
print(score_lin)

0.987971103732035


In [211]:
# Support Vector Machines - note: takes a ridiculous amount of time for large datasets. Ours isn't that large though.
# https://datascience.stackexchange.com/questions/989/svm-using-scikit-learn-runs-endlessly-and-never-completes-executio
SVR = SVR()
SVR.fit(X_train, Y_train)
Y_pred = SVR.predict(X_test)
score_SVR = SVR.score(X_test, Y_test)
score_SVR


0.9831000506179806

In [212]:
# K Nearest Neighbors

knn = KNeighborsRegressor(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
score_knn = knn.score(X_test, Y_test)
print(score_knn)

0.9850217122287265


In [213]:
# Linear SVR (linear suppor vector regression)

linear_SVR = LinearSVR()
linear_SVR.fit(X_train, Y_train)
Y_pred = linear_SVR.predict(X_test)
score_linear_SVR = linear_SVR.score(X_test, Y_test)
print(score_linear_SVR)

0.9876279713415251


In [214]:
# Stochastic Gradient Descent

sgd = SGDRegressor()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
score_sgd = sgd.score(X_test, Y_test)
print(score_sgd)


0.9856548936253497


/Users/arjunrao/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [215]:
# Decision Tree

decision_tree = DecisionTreeRegressor()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)

score_decision_tree = decision_tree.score(X_test, Y_test)
print(score_decision_tree)

0.9792330423140401


In [216]:
# Random Forest

#Can change n_estimators as desired
random_forest = RandomForestRegressor(n_estimators=30, max_depth=10, random_state=1)

random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)

score_random_forest = random_forest.score(X_test, Y_test)
print(score_random_forest)

good_ypreds_list.append(Y_pred)

0.9843053792874018


In [217]:
#show models
models = pd.DataFrame({
    'Model': ['KNN', 'Logistic Regression', 
              'Random Forest', 
              'Stochastic Gradient Decent', 'Linear SVR', 
              'Decision Tree'],
    'Score': [score_knn, score_lin, 
              score_random_forest,
              score_sgd, score_linear_SVR, score_decision_tree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
1,Logistic Regression,0.987971
4,Linear SVR,0.987628
3,Stochastic Gradient Decent,0.985655
0,KNN,0.985022
2,Random Forest,0.984305
5,Decision Tree,0.979233


Clearly all of our models do extremely well in terms of predicting spy_close_price based on signal.
<br>
Logistic regression seems to be the best, but not by a significant margin; we will pursue this further in the regression_models ipynb.
<br>
One type of model we didn't use here was a perceptron or neural network; we explore a neural_network model in the simple_neural_network ipynb.